In [7]:
!uv add openai-agents

Resolved 164 packages in 0.65ms
Audited 160 packages in 1ms


In [4]:
!openai-agents --version

/bin/bash: line 1: openai-agents: command not found


In [11]:
!uv remove openai

Resolved 164 packages in 78ms                                        
Audited 160 packages in 1ms


In [12]:
!uv add openai-agents

Resolved 164 packages in 0.68ms
Audited 160 packages in 1ms


In [19]:
import requests

def get_page_content(url: str) -> str:
    """
    Fetch the textual content of a webpage using the Jina AI Reader service.

    This function prefixes the provided URL with Jina AI's reader endpoint 
    (`https://r.jina.ai/`), sends a GET request to that endpoint, and returns 
    the UTF-8 decoded content of the response.

    Args:
        url (str): The URL of the webpage to fetch.

    Returns:
        str: The decoded UTF-8 content of the requested page.

    Raises:
        requests.RequestException: If an error occurs during the HTTP request.
    """
    reader_url_prefix = 'https://r.jina.ai/'
    request_url = reader_url_prefix + url
    response = requests.get(request_url)
    response.raise_for_status()  # optional but recommended for robustness
    return response.content.decode('utf-8')


In [18]:
get_page_content('https://datatalks.club')

"Title: Welcome to DataTalks.Club\n\nURL Source: https://datatalks.club/\n\nMarkdown Content:\nWelcome to DataTalks.Club\n\n===============\n\nAI Dev Tools Zoomcamp: Learn AI-powered coding assistants and agents[Register here!](https://airtable.com/appJRFiWKHBgmEt70/shrpw7rk55Ewr1jCG)\n\nDataTalks.Club\n--------------\n\n[Articles](https://datatalks.club/articles.html)[Slack](https://datatalks.club/slack.html)[Events](https://datatalks.club/events.html)[Podcast](https://datatalks.club/podcast.html)[Books](https://datatalks.club/books.html)[Courses](https://datatalks.club/blog/guide-to-free-online-courses-at-datatalks-club.html)\n\n* * *\n\nThe place to talk about data\n============================\n\nGlobal online community of data science professionals, ML engineers, and AI practitioners\n-----------------------------------------------------------------------------------------\n\nSubscribe to our weekly newsletter and join our Slack.\n\n We'll keep you informed about everything happen

In [21]:
from agents import Agent, function_tool

In [25]:
assistant_instruction="""
You're a helpful assistant that helps answer user questions
"""

assistant = Agent(
    name='assistant',
    tools=[function_tool(get_page_content)],
    instructions = assistant_instruction,
    model='gpt-4o-mini'
)

In [26]:
from agents import Runner
runner = Runner()

In [27]:
user_prompt = "Summarize the content of https://openai.github.io/openai-agents-python/"
result = await runner.run(assistant, input=user_prompt)

In [31]:
result.new_items[-1].raw_item.content[0].text

'The **OpenAI Agents SDK** is designed to facilitate the creation of agentic AI applications using a simple and effective Python package. Here\'s a summary of its key components:\n\n### Overview\n- **Purpose**: The SDK aims to build agent-oriented AI applications, enhancing usability with minimal abstractions. It serves as a production-ready evolution of prior frameworks, specifically designed to enable developers to work effectively with Large Language Models (LLMs).\n- **Key Features**:\n  - **Agents**: LLMs equipped with specific instructions and tools.\n  - **Handoffs**: Mechanism for agents to delegate tasks to each other.\n  - **Guardrails**: Validations for agent inputs and outputs.\n  - **Sessions**: Automatic management of conversation histories across agent interactions.\n\n### Advantages\nThe SDK is designed with two main principles:\n1. **Simplicity**: Enough features to be useful yet straightforward enough to learn quickly.\n2. **Customizability**: Works well out-of-the-bo

In [32]:
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIAgentsSDKRunner

chat_interface = IPythonChatInterface()
runner = OpenAIAgentsSDKRunner(
    chat_interface = chat_interface,
    agent = assistant
)

In [34]:
await runner.run();

You: Summarize the content of https://www.stromnetz.berlin/


You: stop


Chat ended.


In [37]:
from youtube_transcript_api import YouTubeTranscriptApi

def format_timestamp(seconds:float) -> str:
    """Convert seconds to H:MM:SS if > 1hour, else M:SS"""
    total_seconds = int(seconds)
    hours, remainder = divmod(total_seconds, 3600)
    minutes,secs = divmod(remainder, 60)

    if hours>0:
        return f"{hours}:{minutes:02}:{secs:02}"
    else:
        return f"{minutes}:{secs:02}"

def make_subtitles (transcript) ->str:
    lines=[]

    for entry in transcript:
        ts = format_timestamp(entry.start)
        text = entry.text.replace('\n',' ')
        lines.append(ts+' '+text)
    return '\n'.join(lines)

def fetch_transcript_raw(video_id):
    ytt_api = YouTubeTranscriptApi()
    transcript = ytt_api.fetch(video_id)
    return transcript

def fetch_transcript_text(video_id):
    transcript = fetch_transcript_raw(video_id)
    subtitles = make_subtitles(transcript)
    return subtitles